In [3]:
from collections import defaultdict
from collections import Counter
from ete3 import PhyloTree
import numpy as np
from tqdm import tqdm
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
import time
import csv
import copy
import pandas as pd
from glob import glob
from itertools import combinations

In [4]:
from ete3 import Tree

Load gene families

In [5]:
Fams,species={},[]
for line in open('../gene_family_ressource/orthologous_groups.txt'):
    if line.startswith("#"): continue
    fid,glt=line.strip().split('\t',1)
    gbs=defaultdict(list)
    for g in glt.split():
        spc,gid=g.split('_',1)
        gbs[spc].append(gid)
    species.extend(gbs.keys())
    Fams[fid]=gbs
        #if i==5: break
species=list(set(species))

In [12]:
print(list(Fams.items())[0])
print(len(Fams))
print(sorted(species))

('OG_1', defaultdict(<class 'list'>, {'Bralan': ['BL09207'], 'Strpur': ['LOC591688'], 'Brabel': ['204820R'], 'Braflo': ['BF14516'], 'Danrer': ['ENSDARG00000038505'], 'Sackow': ['Sakowv30028108m'], 'Orylat': ['ENSORLG00000023347'], 'Xentro': ['polr2h'], 'Parata': ['EA33602'], 'Chipun': ['Chipu0001179'], 'Musmus': ['ENSMUSG00000021018'], 'Homsap': ['ENSG00000163882'], 'Petmar': ['POLR2H'], 'Letrei': ['Hic_chr_27_270', 'Hic_chr_27_271'], 'Lepocu': ['ENSLOCG00000008946'], 'Latcha': ['ENSLACG00000015017'], 'Acapla': ['LOC110983881'], 'Calmil': ['SINCAMG00000007000'], 'Galgal': ['ENSGALG00000008534'], 'Letjap': ['g12991']}))
22691
['Acapla', 'Brabel', 'Braflo', 'Bralan', 'Calmil', 'Chipun', 'Danrer', 'Eptbur', 'Galgal', 'Homsap', 'Latcha', 'Lepocu', 'Letjap', 'Letrei', 'Musmus', 'Orylat', 'Parata', 'Petmar', 'Ptyfla', 'Sackow', 'Strpur', 'Xentro']


Creates non-redundant species set

In [76]:
print(Fams['OG_22003'])

defaultdict(<class 'list'>, {'Strpur': ['LOC105440589'], 'Acapla': ['LOC110984772']})


Load species tree

In [9]:
spT=PhyloTree(open('deut2.tre').read(), format=1)

In [ ]:
['Strpur','Acapla','Sackow','Ptyfla']

In [16]:
print(spT)


         /-Strpur
      /-|
     |   \-Acapla
   /-|
  |  |   /-Sackow
  |   \-|
  |      \-Ptyfla
  |
  |      /-Brabel
--|   /-|
  |  |  |   /-Braflo
  |  |   \-|
  |  |      \-Bralan
  |  |
  |  |            /-Letjap
  |  |         /-|
   \-|      /-|   \-Letrei
     |     |  |
     |   /-|   \-Petmar
     |  |  |
     |  |  |   /-Parata
     |  |   \-|
     |  |      \-Eptbur
      \-|
        |      /-Calmil
        |   /-|
        |  |   \-Chipun
        |  |
        |  |         /-Lepocu
         \-|      /-|
           |     |   \-Amical
           |   /-|
           |  |  |   /-Danrer
           |  |   \-|
            \-|      \-Orylat
              |
              |   /-Latcha
               \-|
                 |   /-Xentro
                  \-|
                    |   /-Galgal
                     \-|
                       |   /-Homsap
                        \-|
                           \-Musmus


### Determine phylostrata of  genes 

In [7]:
print(Fams['OG_171'])

defaultdict(<class 'list'>, {'Bralan': ['BL14681'], 'Strpur': ['LOC575087'], 'Ptyfla': ['pfl_40v0_9_20150316_1g2352'], 'Brabel': ['142360R'], 'Braflo': ['BF05005'], 'Danrer': ['ENSDARG00000102643'], 'Sackow': ['Sakowv30013968m'], 'Eptbur': ['Eptbu0013095'], 'Orylat': ['ENSORLG00000015261'], 'Xentro': ['chmp1a'], 'Parata': ['EA65120'], 'Chipun': ['Chipu0006856'], 'Musmus': ['ENSMUSG00000000743'], 'Homsap': ['ENSG00000131165'], 'Petmar': ['CHMP1A'], 'Letrei': ['Hic_chr_52_117'], 'Lepocu': ['ENSLOCG00000003012'], 'Latcha': ['ENSLACG00000014862'], 'Acapla': ['LOC110975658'], 'Calmil': ['SINCAMG00000003614'], 'Galgal': ['ENSGALG00000006161'], 'Letjap': ['g15058']})


In [8]:
PhyloSt=defaultdict(int)
g2f={}
genephylo={}
tor=defaultdict(int)
size=[]
for fid in Fams:
    gbs=Fams[fid]
    spset=[s for s,gbs in gbs.items() if len(gbs)>0]
    phtyp=spT.get_common_ancestor(spset) if len(spset)>1 else spT.search_nodes(name=spset[0])
    #print(fid,len(phtyp))
    tor[phtyp.name]+=1
    size.append(len(spset))
    for g in gbs['Parata']:
        PhyloSt[phtyp.name]+=1
        genephylo[g]=phtyp.name
        g2f[g]=fid
        

In [16]:
with open('gene_strata.txt','w') as out:
    for g,s in genephylo.items():
        out.write("{}\t{}\n".format(g,s))

In [9]:
print(PhyloSt,sum(PhyloSt.values()))

defaultdict(<class 'int'>, {'Deuterostomia': 17189, 'Chordata': 513, 'Vertebrata': 1338, 'Myxines': 3263, 'Cyclostomata': 464}) 22767


In [10]:
Counter(size)

Counter({20: 1378,
         22: 2389,
         16: 324,
         21: 2560,
         15: 415,
         18: 573,
         5: 625,
         7: 629,
         19: 812,
         3: 2947,
         2: 6112,
         8: 304,
         6: 432,
         12: 269,
         11: 241,
         4: 786,
         10: 511,
         13: 277,
         14: 353,
         17: 431,
         9: 323})

## PANTHER

In [15]:
Pthr=defaultdict(dict)
for fp in glob("panthers/*.pthr"):
    sp=fp.split('/')[1].replace('.pthr','')
    pps=defaultdict(list)
    for r in csv.reader(open(fp),delimiter='\t'):
        #print(r)
        pps[r[1]].append(r[0].split('_',1)[1])
        #print(pps)
        #break
    for pth in pps:
        #pthr[pth]
        Pthr[pth][sp]=pps[pth]
        
    #print(list(pps.items())[0])
    #print(sp)

In [18]:
origins=defaultdict(int)
losses=defaultdict(int)
idGainLoss={}
for fam in Pthr.items():
    fid,nspec,ndesc,oritax,lost=checkFam(fam,spT)
    idGainLoss[fid]=(nspec,ndesc,oritax,lost)
    origins[oritax]+=1
    for tax in lost:
        losses[tax]+=1
        

In [10]:
spTi=copy.deepcopy(spT)
for node in spTi.traverse("postorder"):
    #print node.name,origins[node.name],losses[node.name]
    #node.add_features(origins=origins[node.name],losses=losses[node.name])
    node.name="{0}_{1}_{2}".format(node.name,origins[node.name],losses[node.name])

In [28]:
spTl=copy.deepcopy(spT)
for node in spTl.traverse("postorder"):
    #print node.name,origins[node.name],losses[node.name]
    #node.add_features(origins=origins[node.name],losses=losses[node.name])
    if not node.is_leaf():
        node.name="{0}".format(losses[node.name])


In [63]:
PthNms={}
for r in csv.reader(open('/Users/fmarletaz/Dropbox (UCL)/Genomes/Hagfish/Hagfish_genome/gene_family_1/panther.names.tab'),delimiter='\t'):
    pth=r[0].replace('.mod','').replace('.mag','').replace('.',':')
    pnm=r[1][0]+r[1][1:].lower()
    PthNms[pth]=pnm

In [65]:
igl=[[s,len(sum(list(Pthr[s].values()),[])),v[0],v[1],v[2],','.join(v[3]),PthNms.get(s,'Not found')] for s,v in idGainLoss.items()]

In [66]:
fidGL=pd.DataFrame(igl, columns = ['FID', 'NbGenes','NbSpecies','NbDesc','Strata','Losses','Panther Name'])  

In [68]:
fidGL.to_csv("Panther_GainLoss.tsv",index=False,sep='\t')

In [20]:
print(spTl.get_ascii(show_internal=True))


          /-Strpur
      /3202
     |    \-Acapla
   /0|
  |  |    /-Sackow
  |   \1790
  |       \-Ptyfla
  |
  |       /-Brabel
-0|   /2976
  |  |   |   /-Braflo
  |  |    \705
  |  |       \-Bralan
  |  |
  |  |                /-Letjap
  |  |            /1147
   \0|        /2485   \-Letrei
     |       |   |
     |    /2196   \-Petmar
     |   |   |
     |   |   |    /-Parata
     |   |    \3844
     |   |        \-Eptbur
      \1881
         |        /-Calmil
         |    /3169
         |   |    \-Chipun
         |   |
         |   |           /-Lepocu
          \1679      /2649
             |      |    \-Amical
             |   /2549
             |  |   |   /-Danrer
             |  |    \932
              \796      \-Orylat
                |
                |    /-Latcha
                 \1423
                    |   /-Xentro
                     \795
                       |    /-Galgal
                        \1322
                           |    /-Homsap
                     

In [ ]:
print(spTi.get_ascii(show_internal=True))

In [29]:
print(spTl.write(format=1))

(((Strpur:1,Acapla:1)3202:1,(Sackow:1,Ptyfla:1)1790:1)0:1,((Brabel:1,(Braflo:1,Bralan:1)705:1)2976:1,((((Letjap:1,Letrei:1)1147:1,Petmar:1)2485:1,(Parata:1,Eptbur:1)3844:1)2196:1,((Calmil:1,Chipun:1)3169:1,(((Lepocu:1,Amical:1)2649:1,(Danrer:1,Orylat:1)932:1)2549:1,(Latcha:1,(Xentro:1,(Galgal:1,(Homsap:1,Musmus:1)1072:1)1322:1)795:1)1423:1)796:1)1679:1)1881:1)0:1);


In [21]:
for pth in pthr:
    print(pth,pthr[pth])
    break

PTHR19346:SF2 {'Eptbur': ['Eptbu0029334', 'Eptbu0019791'], 'Lepocu': ['ENSLOCG00000011125'], 'Petmar': ['LOC116947717', 'SLC35F3', 'SLC35F4'], 'Musmus': ['ENSMUSG00000021852'], 'Homsap': ['ENSG00000151812'], 'Galgal': ['ENSGALG00000012094'], 'Letjap': ['g17577', 'g4205', 'g8559', 'g31672', 'g26173'], 'Chipun': ['Chipu0019746', 'Chipu0002082', 'Chipu0033669'], 'Xentro': ['slc35f4-like'], 'Letrei': ['Hic_chr_14_141', 'Hic_chr_41_266'], 'Calmil': ['SINCAMG00000016050', 'SINCAMG00000016049'], 'Latcha': ['ENSLACG00000018940'], 'Parata': ['EA55570', 'EA58774'], 'Danrer': ['ENSDARG00000077622', 'ENSDARG00000103124']}


## Testing family expansions

In [15]:
#only testing species expansion
def fishExp(gbs,tbs):
    cbs=dict((sp,len(gs)) for sp,gs in gbs.items())
    med=np.median(list(cbs.values()))
    #print(cbs,med)
    spval={}
    for sp,st in cbs.items():
        nis=sum(cbs.values())-st
        nit=sum(tbs.values())-tbs[sp]-sum(cbs.values())+st
        odds,pval=stats.fisher_exact([[st, tbs[sp]-st], [nis, nit]],alternative='greater')
        nmed=np.median([cbs[ssp] for ssp in cbs if not ssp==sp])
        tmed=np.median([tbs[ssp] for ssp in tbs if not ssp==sp])
        m_odds,m_pval=stats.fisher_exact([[st, tbs[sp]-st], [nmed, tmed]],alternative='greater')
        #print(sp,pval,[st, tbs[sp]-st], [nis, nit],m_pval,[st, tbs[sp]-st], [nmed, tmed])
        spval[sp]=(cbs[sp],m_pval)

    return spval

calculate number of genes per species

In [16]:
tbs=defaultdict(int)
for fid,gbs in Fams.items():
    for sp in gbs:
        tbs[sp]+=len(gbs[sp])

In [17]:
toosmall=0
expRes=[]
for fid,gbs in tqdm(Fams.items()):
    cbs=dict((sp,len(gs)) for sp,gs in gbs.items())
    if len(cbs.keys())<3 or sum(cbs.values())<5:
        toosmall+=1
        continue
    try:
        spval=fishExp(gbs,tbs)
        expRes.append((fid,spval))
    except:
        print(fid,cbs)

100%|██████████| 22691/22691 [02:05<00:00, 180.52it/s]


In [21]:
print(len(Fams),len(expRes),toosmall)

22691 14513 8178


In [22]:
spFam,spPval=defaultdict(list),defaultdict(list)
for fid,spval in expRes:
    for sp in spval:
        ct,pval=spval[sp]
        spFam[sp].append(fid)                     
        spPval[sp].append(pval)

In [23]:
famEnrich=defaultdict(dict)
for sp in spPval:
    #signif_pvals = sidak(spPval[sp], alpha=0.05)
    #before_adj=['True' if p < 0.05  else 'False' for p in spPval[sp] ]
    #1signif_pvals = lsu(np.array(spPval[sp]), q=0.05) #this is benferroni-hochberg
    adj=multipletests(pvals=np.array(spPval[sp]), alpha=0.05, method="fdr_bh")
    print (sp,Counter(adj[0]))
    for fam,pth,pval in zip(spFam[sp],adj[0],adj[1]):
        famEnrich[fam][sp]=(pth,pval)
        


Bralan Counter({False: 10623, True: 93})
Strpur Counter({False: 9314, True: 127})
Brabel Counter({False: 10511, True: 64})
Braflo Counter({False: 10247, True: 69})
Danrer Counter({False: 10770, True: 29})
Sackow Counter({False: 9868, True: 107})
Orylat Counter({False: 10502, True: 20})
Xentro Counter({False: 10632, True: 55})
Parata Counter({False: 14478, True: 35})
Chipun Counter({False: 10874, True: 6})
Musmus Counter({False: 10903, True: 32})
Homsap Counter({False: 10896, True: 17})
Petmar Counter({False: 9551, True: 11})
Letrei Counter({False: 5411, True: 43})
Lepocu Counter({False: 10755, True: 8})
Latcha Counter({False: 10597, True: 17})
Acapla Counter({False: 9401, True: 43})
Calmil Counter({False: 10356, True: 11})
Galgal Counter({False: 10015, True: 20})
Letjap Counter({False: 9558, True: 15})
Ptyfla Counter({False: 8612, True: 153})
Eptbur Counter({False: 8931, True: 50})


In [ ]:
for g in Fams['OG_191']['Parata']:
    print(g,Panther.get(g))

In [23]:
HsaNames={e[0]:e[1] for e in csv.reader(open('Hsa_mart_info.txt'),delimiter='\t')}

In [24]:
Panther={e[0]:e[1].split(':')[0] for e in csv.reader(open('Pata_pep.pthr'),delimiter='\t')}

In [ ]:
print(Panther)

In [25]:
Names={e[13]:e[12] for e in csv.reader(open('/Users/fmarletaz/Dropbox/Genomes/Hagfish/Hagfish_data_rename/Pata_MYAQb_rnm_ah2p.bed'),delimiter='\t')}

In [26]:
Pfam={}
for l in open('Pata_ah2p.pfam'):
    if not l.strip(): continue
    e=l.split()
    #print(e)
    Pfam[e[0]]=e[5].split('.')

In [27]:
panthNames={}
for e in csv.reader(open('panther.names.tab'),delimiter='\t'):
    pth=e[0].split('.')[0]
    if not pth in panthNames:
        panthNames[pth]=e[1][0]+e[1][1:].lower()

In [56]:
enrList=[]
for fid,gbs in Fams.items():
    cbs=dict((sp,len(gs)) for sp,gs in gbs.items())
    pvl=famEnrich[fid]
    plnm=[Names.get(g,'-') for g in gbs.get('Parata',[])]
    plnm=[d for d in plnm if not d.startswith('unchar') and not d=='-']
        #print(plnm)
    hsnm=[HsaNames[g] for g in gbs.get('Homsap',[]) if g in HsaNames]
    pfam=set(list(sum([Pfam[g] for g in gbs.get('Parata',[]) if g in Pfam],[])))
    #print(pfam)
    panther=set([Panther[g] for g in gbs.get('Parata',[]) if g in Panther])
    pthNames=set([panthNames.get(p,'') for p in panther]) 
    enriched= [(s,p[1]) for s,p in pvl.items() if p[0]==True]
    if len(enriched)==0:
        continue
    #print(enriched)
    outList=[fid,len(cbs),sum(cbs.values()),np.median(list(cbs.values())),','.join([s for s,p in enriched]),
            ','.join([str(cbs[s]) for s,p in enriched]),','.join(plnm),
             ','.join(hsnm),','.join(['\'{}\''.format(g) for g in gbs['Parata']]),','.join(set(panther)),','.join(pthNames),','.join(set(pfam)),','.join(['{:.2e}'.format(float(p)) for s,p in enriched])]
    enrList.append(outList)  
        
    

In [33]:
print(pfam)

set()


In [57]:
enrichPD=pd.DataFrame(enrList, columns = ['FID', 'NbSp','NbGenes','MedGeneNb','Enriched','NbEnriched','Pliv_names','Hsa_names','Pliv_id','Panther','PthrNames','PFAM','pvals'])  

In [13]:
enrichPD[enrichPD.Enriched.str.contains('Parata')].to_csv('enriched_in_Parata.txt',sep='\t')

NameError: name 'enrichPD' is not defined

In [ ]:
enrichPD[enrichPD.Enriched.str.contains('Parata')]

## Gain and losses

In [16]:
from tqdm import tqdm

In [6]:
def checkFam(fam,spTd):
    fid,gbs=fam
    spset=[s for s,gbs in gbs.items() if len(gbs)>0]

    #print(phtyp.name)
    #phyloCt[phtyp.name]+=1
    #gbs[Ai]
    #spTp=copy.deepcopy(spTd)
    for leaf in spTd:
        pv='1' if leaf.name in gbs else '0'
        leaf.add_features(presence=pv)
    phtyp=spTd.get_common_ancestor(spset) if len(spset)>1 else spT.search_nodes(name=spset[0])[0]
    ndesc=len([l.name for l in phtyp.get_leaves()])
    lost=[node.name  for node in phtyp.get_monophyletic(values=['0'], target_attr="presence")]
    #print spTd.get_ascii(attributes=["name","presence"], show_internal=True)
    #for node in spTd.get_monophyletic(values=['0'], target_attr="presence"):
    #    print node.name
    #    print node.get_ascii(attributes=["presence", "name"], show_internal=False)
    return fid,len(gbs.keys()),ndesc,phtyp.name,lost



In [4]:
def checkFamCounts(fam,spTd):
    fid,cbs=fam
    spset=[s for s,n in cbs.items() if n>0]
    if len(spset)>1:
        phtyp=spT.get_common_ancestor(spset)
    else:
        phtyp=[l for l in spT.get_leaves() if l.name==spset[0]][0]
    #print phtyp.name
    #phyloCt[phtyp.name]+=1
    #gbs[Ai]
    ndesc=len([l.name for l in phtyp.get_leaves()])
    for leaf in spTd:
        pv='1' if leaf.name in spset else '0'
        leaf.add_features(presence=pv)
    lost=[node.name  for node in phtyp.get_monophyletic(values=['0'], target_attr="presence")]
    #print spTd.get_ascii(attributes=["name","presence"], show_internal=True)
    #for node in spTd.get_monophyletic(values=['0'], target_attr="presence"):
    #    print node.name
    #    print node.get_ascii(attributes=["presence", "name"], show_internal=False)
    return fid,len(spset),ndesc,phtyp.name,lost




In [ ]:
checkFam(list(Fams.items())[555],spT)

In [89]:
print(list(Fams.items())[555])

('OG_559', defaultdict(<class 'list'>, {'Linana': ['LANA25934'], 'Bralan': ['BL04443'], 'Strpur': ['LOC592736'], 'Parliv': ['PL21635'], 'Braflo': ['BL153011'], 'Sackow': ['Sakowv30029968m'], 'Lytvar': ['L_var_26160-RA'], 'Homsap': ['ENSG00000163291'], 'Patyes': ['PY_T06293'], 'Apojap': ['BSL78_00950'], 'Limpol': ['XP_013789801', 'XP_013790183'], 'Captel': ['CapteG138960'], 'Patmin': ['PMI_028803-RA', 'PMI_014526-RA'], 'Lepocu': ['ENSLOCG00000007313'], 'Acapla': ['LOC110976640'], 'Calmil': ['SINCAMG00000004329', 'SINCAMG00000015047'], 'Lotgig': ['LotgiG144778'], 'Molocc': ['g07092'], 'Nemvec': ['NEMVEDRAFT_v1g227246'], 'Tricas': ['TC000446'], 'Dromel': ['FBgn0038256'], 'Cioint': ['ENSCING00000018091', 'ENSCING00000019528'], 'Annjap': ['LOC117103605'], 'Astrub': ['AR14388'], 'Ptyfla': ['pfl_40v0_9_20150316_1g11072']}))


In [99]:
print(Fams['OG_22003'])

defaultdict(<class 'list'>, {'Strpur': ['LOC105440589'], 'Acapla': ['LOC110984772']})


In [ ]:
origins=defaultdict(int)
losses=defaultdict(int)
idGainLoss={}
for fam in tqdm(Fams.items()):
    fid,nspec,ndesc,oritax,lost=checkFam(fam,spT)
    idGainLoss[fid]=(nspec,ndesc,oritax,lost)
    origins[oritax]+=1
    for tax in lost:
        losses[tax]+=1
        

In [25]:
losses['Letrei']=495

In [26]:
print(losses)

defaultdict(<class 'int'>, {'Ptyfla': 1873, 'Eptbur': 414, 'Amical': 10961, 'Sackow': 666, 'Xentro': 901, 'Letrei': 495, 'Galgal': 1253, 'Cyclostomata': 836, 'Holostei': 779, 'Braflo': 1029, 'Homsap': 144, 'Echinodermata': 1055, 'Parata': 489, 'Calmil': 1141, 'Vertebrata': 1120, 'Orylat': 671, 'Brabel': 479, 'Chondrichthyes': 719, 'Sarcopterygia': 296, 'Letjap': 274, 'Latcha': 1245, 'Acapla': 523, 'Hyperoartia': 630, 'Myxines': 1415, 'Strpur': 555, 'Hemichordata': 467, 'Osteichthyes': 278, 'Petmar': 887, 'Gnathostomata': 374, 'Tetrapoda': 291, 'Chipun': 410, 'Euteleostei': 376, 'Mammalia': 268, 'Cephalochordata': 694, 'Leth': 447, 'Actinopterygii': 617, 'Danrer': 388, 'Musmus': 109, 'Amniotes': 403, 'Ceph2': 129, 'Bralan': 414})


In [30]:
spTi=copy.deepcopy(spT)
for n in spTi.traverse("postorder"):
    nm=n.name
    n.dist="{}".format(origins[nm]/sum(origins.values()))
    #print node.name,origins[node.name],losses[node.name]
    if not n.is_leaf():
        n.name="{}".format(origins[nm])
    #n.add_features(origins=origins[node.name],losses=losses[node.name])
    #node.name="{0}_{1}_{2}_{3}".format(node.name,origins[node.name],losses[node.name],totdups[node.name])

In [ ]:
spTi=copy.deepcopy(spT)
for n in spTi.traverse("postorder"):
    nm=n.name
    n.dist="{}".format(losses[nm]/sum(losses.values()))
    #print node.name,origins[node.name],losses[node.name]
    if not n.is_leaf():
        n.name="{}".format(losses[nm])
    #n.add_features(origins=origins[node.name],losses=losses[node.name])
    #node.name="{0}_{1}_{2}_{3}".format(node.name,origins[node.name],losses[node.name],totdups[node.name])

In [ ]:
spTd=deepcopy(RTL.spTree)
for l in spTd.traverse("postorder"):
    nm=l.name
    l.dist="{}".format(dupsCt[nm]/sum(dupsCt.values()))
    if not l.is_leaf():
        l.name="{}".format(dupsCt[nm])

In [ ]:
print(spTi)

In [121]:
spTi=copy.deepcopy(spT)
for node in spTi.traverse("postorder"):
    #print node.name,origins[node.name],losses[node.name]
    #node.add_features(origins=origins[node.name],losses=losses[node.name])
    node.name="{0}_{1}_{2}_{3}".format(node.name,origins[node.name],losses[node.name],totdups[node.name])

In [31]:
print(spTi.write(format=1))

(((Strpur:0,Acapla:0)197:0.00868186,(Sackow:0,Ptyfla:0)630:0.0277643)380:0.0167467,((Brabel:0,(Braflo:0,Bralan:0)352:0.0155128)2518:0.110969,((((Letjap:0,Letrei:0)274:0.0120753,Petmar:0)605:0.0266626,(Parata:0,Eptbur:0)2288:0.100833)416:0.0183333,((Calmil:0,Chipun:0)68:0.00299678,(((Lepocu:0,Amical:0)0:0,(Danrer:0,Orylat:0)109:0.00480367)110:0.00484774,(Latcha:0,(Xentro:0,(Galgal:0,(Homsap:0,Musmus:0)547:0.0241065)60:0.00264422)73:0.00321713)45:0.00198317)213:0.00938698)1122:0.0494469)1310:0.0577321)551:0.0242828);


In [ ]:
print(spTi.get_ascii(show_internal=True))

In [19]:
igl=[[s,len(sum(list(Fams[s].values()),[])),v[0],v[1],v[2],','.join(v[3])] for s,v in idGainLoss.items()]

In [20]:
fidGL=pd.DataFrame(igl, columns = ['FID', 'NbGenes','NbSpecies','NbDesc','Strata','Losses'])  

In [21]:
fidGL

,FID,NbGenes,NbSpecies,NbDesc,Strata,Losses
0,OG_1,21,20,23,Deuterostomia,"Ptyfla,Eptbur,Amical"
1,OG_2,23,22,23,Deuterostomia,Amical
2,OG_3,20,20,23,Deuterostomia,"Sackow,Amical,Xentro"
3,OG_4,31,20,23,Deuterostomia,"Letrei,Amical,Galgal"
4,OG_5,20,16,23,Deuterostomia,"Cyclostomata,Holostei"
...,...,...,...,...,...,...
22686,OG_22687,2,2,2,Myxines,
22687,OG_22688,2,2,2,Myxines,
22688,OG_22689,2,2,2,Myxines,
22689,OG_22690,2,2,2,Myxines,


In [28]:
plnp={fid:','.join([Names[g] for g in gbs.get('Parata',[]) if g in Names]) for fid,gbs in Fams.items()}

In [39]:
xtid={fid:','.join([g for g in gbs.get('Xentro',[])]) for fid,gbs in Fams.items()}

In [40]:
loid={fid:','.join([g for g in gbs.get('Lepocu',[])]) for fid,gbs in Fams.items()}

In [60]:
pmid={fid:','.join([g for g in gbs.get('Petmar',[])]) for fid,gbs in Fams.items()}

In [30]:
hsnm={fid:','.join([HsaNames.get(g,'-') for g in gbs.get('Homsap',[])]) for fid,gbs in Fams.items()}

In [31]:
panth={fid:','.join(set([Panther.get(g,'') for g in gbs.get('Parata',[])])) for fid,gbs in Fams.items()}

In [32]:
print(list(panth.items())[0])

('OG_1', 'PTHR10917')


In [38]:
sum([Panther.get('PL40550',['-']) for g in gbs.get('Parliv',[])],[])

[]

In [ ]:
pfam={fid:','.join(set(sum([Pfam.get(g,['-']) for g in gbs.get('Parata',[])],[]))) for fid,gbs in Fams.items()}

In [96]:
len(sum(list(Fams['OG_1'].values()),[]))

36

In [ ]:
print(pfam)

In [33]:
fidGL['Eata_names']=fidGL['FID'].map(plnp) 

In [34]:
fidGL['Hsa_names']=fidGL['FID'].map(hsnm) 

In [61]:
fidGL['Pma_ids']=fidGL['FID'].map(pmid) 

In [35]:
fidGL['gids']=fidGL['FID'].map(plid) 

In [41]:
fidGL['Gar_ids']=fidGL['FID'].map(loid) 

In [42]:
fidGL['Frog_ids']=fidGL['FID'].map(xtid) 

In [ ]:
fidGL['Pmar_ids']=fidGL['FID'].map(xtid) 

In [113]:
fidGL['Duplications']=fidGL['FID'].map(dupNodes) 

In [36]:
fidGL['Panther']=fidGL['FID'].map(panth) 

In [ ]:
fidGL['PFAM']=fidGL['FID'].map(pfam) 

In [43]:
fidGL

,FID,NbGenes,NbSpecies,NbDesc,Strata,Losses,Eata_names,Hsa_names,gids,Panther,Gar_ids,Frog_ids
0,OG_1,21,20,23,Deuterostomia,"Ptyfla,Eptbur,Amical",POLR2H,POLR2H,EA33602,PTHR10917,ENSLOCG00000008946,polr2h
1,OG_2,23,22,23,Deuterostomia,Amical,COPB1-1,COPB1,EA45752,PTHR10635,ENSLOCG00000003198,copb1
2,OG_3,20,20,23,Deuterostomia,"Sackow,Amical,Xentro",YIPF4,YIPF4,EA48129,PTHR21236,ENSLOCG00000016640,
3,OG_4,31,20,23,Deuterostomia,"Letrei,Amical,Galgal","SYMPK-2,SYMPK-3,SYMPK-1,SYMPK-4",SYMPK,"EA59309,EA59730,EA58135,EA60811",PTHR15245,"ENSLOCG00000014229,ENSLOCG00000014228",sympk
4,OG_5,20,16,23,Deuterostomia,"Cyclostomata,Holostei",,"SYNJ2BP-COX16,SYNJ2BP",,,,synj2bp
...,...,...,...,...,...,...,...,...,...,...,...,...
22686,OG_22687,2,2,2,Myxines,,Unchar_15816,,EA60352,,,
22687,OG_22688,2,2,2,Myxines,,Unchar_18929,,EA72657,,,
22688,OG_22689,2,2,2,Myxines,,Unchar_13670,,EA52681,,,
22689,OG_22690,2,2,2,Myxines,,Unchar_13220,,EA51456,,,


In [125]:
fidGL[(fidGL.Losses.str.contains('Myxines')) & (fidGL.Hsa_names!='') ]#.to_csv('lost_in_osteo.txt',sep='\t')

,FID,NbGenes,NbSpecies,NbDesc,Strata,Losses,Eata_names,Hsa_names,gids,Duplications,Panther
43,OG_44,21,18,23,Deuterostomia,"Letrei,Myxines,Amical,Orylat",,RIOX2,,"Brabel,Braflo,Ceph2",
102,OG_103,32,12,19,Chordata,"Myxines,Chondrichthyes,Amical,Latcha,Galgal",,"TCAF2,TCAF2C,TCAF1",,"Actinopterygii,Hyperoartia,Mammalia,Danrer,Hom...",
127,OG_128,436,20,23,Deuterostomia,"Myxines,Amical",,"NXPE2,NXPE1,NXPE4,NXPE3",,"Brabel,Vertebrata,Petmar,Mammalia,Orylat,Ambul...",
134,OG_135,22,15,23,Deuterostomia,"Hemichordata,Letrei,Myxines,Chipun,Amical,Danrer",,ZADH2,,"Acapla,Chordata,Strpur,Letjap",
148,OG_149,90,18,23,Deuterostomia,"Leth,Myxines,Amical",,"HSD11B1L,HSD11B1",,"Brabel,Sackow,Ptyfla,Braflo,Vertebrata,Orylat,...",
...,...,...,...,...,...,...,...,...,...,...,...
21673,OG_21674,11,10,16,Vertebrata,"Leth,Myxines,Calmil,Amical",,RPL39,,Musmus,
21677,OG_21678,21,12,16,Vertebrata,"Letrei,Myxines,Amical",,GJA5,,"Vertebrata,Hyperoartia,Galgal,Letjap,Euteleostei",
21776,OG_21777,15,10,16,Vertebrata,"Letrei,Myxines,Chipun,Amical,Danrer",,"AC008676.3,FNDC9",,"Homsap,Gnathostomata",
21805,OG_21806,7,5,16,Vertebrata,"Letrei,Myxines,Chipun,Actinopterygii,Latcha,Xe...",,"CTXND2,CTXND1",,Mammalia,


In [66]:
fidGL[(fidGL.Panther!='')]#.to_csv('lost_in_osteo.txt',sep='\t')

,FID,NbGenes,NbSpecies,NbDesc,Strata,Losses,Eata_names,Hsa_names,gids,Duplications,Panther,PFAM
0,OG_1,21,20,23,Deuterostomia,"Ptyfla,Eptbur,Amical",,POLR2H,EA33602,Letrei,PTHR10917,
1,OG_2,23,22,23,Deuterostomia,Amical,,COPB1,EA45752,Sackow,PTHR10635,
2,OG_3,20,20,23,Deuterostomia,"Sackow,Amical,Xentro",,YIPF4,EA48129,,PTHR21236,
3,OG_4,31,20,23,Deuterostomia,"Letrei,Amical,Galgal",,SYMPK,"EA59309,EA59730,EA58135,EA60811","Parata,Lepocu,Gnathostomata,Myxines,Ceph2,Chipun",PTHR15245,
5,OG_6,21,20,23,Deuterostomia,"Braflo,Letrei,Amical",,VTI1B,EA36571,Letjap,PTHR21230,
...,...,...,...,...,...,...,...,...,...,...,...,...
21290,OG_21291,2,2,2,Myxines,,,,EA49706,NaN,PTHR31080,
21296,OG_21297,2,2,2,Myxines,,,,EA67666,NaN,PTHR16423,
21299,OG_21300,2,2,2,Myxines,,,,EA67794,NaN,PTHR14972,
22679,OG_22680,2,2,2,Myxines,,,,EA75053,NaN,PTHR11588,


In [186]:
fidGL[fidGL.Strata.str.contains('Echinodermata')]#.to_csv('novel_in_echinoidea.txt',sep='\t')

,FID,NbGenes,NbSpecies,NbDesc,Strata,Losses,Pliv_names,Hsa_names,Pliv_ids,Duplications,Panther,PFAM
34,OG_35,7,7,8,Echinodermata,Apojap,unchar-8163,,PL34457,,subfamily not named,7tm_2
123,OG_124,3,3,8,Echinodermata,"Acapla,Astrub,Echinoidea",,,,NaN,,
257,OG_260,7,6,8,Echinodermata,"Patmin,Parliv",,,,Echinodermata,,
297,OG_300,3,3,8,Echinodermata,"Patmin,Echinozoa",,,,NaN,,
624,OG_629,6,3,8,Echinodermata,"Patmin,Echinozoa",,,,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...
17746,OG_18403,2,2,8,Echinodermata,"Acapla,Astrub,Echinozoa",,,,NaN,,
17762,OG_18420,2,2,8,Echinodermata,"Acapla,Astrub,Echinozoa",,,,NaN,,
17764,OG_18422,2,2,8,Echinodermata,"Patmin,Astrub,Echinozoa",,,,NaN,,
17765,OG_18423,2,2,8,Echinodermata,"Asteroidea,Apojap,B",unchar-705,,PL32783,NaN,-,-


In [47]:
gar_ids=fidGL[fidGL.Losses.str.contains('Myxines')].Gar_ids#.to_csv('lost_in_osteo.txt',sep='\t')

In [57]:
frg_ids=fidGL[fidGL.Losses.str.contains('Myxines')].Frog_ids#.to_csv('lost_in_osteo.txt',sep='\t')

In [62]:
pma_ids=fidGL[fidGL.Losses.str.contains('Myxines')].Pma_ids#.to_csv('lost_in_osteo.txt',sep='\t')

In [53]:
gar_ids=sum([g.split(',') for g in gar_ids if not g==''],[])

In [58]:
frg_ids=sum([g.split(',') for g in frg_ids if not g==''],[])

In [63]:
pma_ids=sum([g.split(',') for g in pma_ids if not g==''],[])

In [64]:
with open('pmar_list.txt','w') as out: 
    out.write('\n'.join(pma_ids))

In [44]:
fidGL.to_csv('all_families_gain_loss.txt',sep='\t')

## Duplications 

In [19]:
def decoTree(t,outtax):
    for l in t:
        sp,gid=l.name.split('_',1)
        gid=gid.split('.')[0]
        l.add_features(species=sp,gid=gid)
    #first, reset rooting with mid-point rooting
    R = t.get_midpoint_outgroup()
    t.set_outgroup(R)
    #then root with deuterostome ancestor
    sot=[l for l in t if l.species in outtax] 
    if len(sot)>0:
        anc=t.get_common_ancestor(sot)
        if not anc==t:
            t.set_outgroup(anc)
    return t

In [20]:
def age(node,sptr):
    tsp=list(set([lf.species for lf in node]))
    if len(tsp)==1:
        return tsp[0]
    else:
        tanc=sptr.get_common_ancestor(tsp).name
        return tanc

In [21]:
outgroups=['Strpur','Acapla','Sackow','Ptyfla']

In [22]:
rTrees={}
nclgbfl=[]
strees={}
nexcl=0
for l in open('../../Gene_families/brofams_0221_rec.trees'):
    o,ts=l.strip().split(':',1)
    og=o.split('/')[-1].rsplit('_',1)[0]
    #print(og)
    strees[og]=ts
    t=Tree(ts, format=1)
    try:
        t=decoTree(t,outgroups)
        spl=list(set([l.species for l in t]))
    #print(spl)
        if len(spl)<3:
            nexcl+=1
            continue
        rTrees[og]=t
    except:
        print("troubles with {}".format(og))

In [19]:
print(nexcl,len(rTrees))

0 12557


In [23]:
dupNodes={}
totdups=defaultdict(int)
for f,st in rTrees.items():
    dupnds=[sst for sst in st.search_nodes(D='Y')]
    ages=[age(n,spT) for n in dupnds]
    for a in ages:
        totdups[a]+=1
    dupNodes[f]=','.join(list(set(ages)))
    #print(f,dupnds,ages)
    
        

In [17]:
st=rTrees['OG_1000']

In [ ]:
dupnds=[sst for sst in st.search_nodes(D='Y')]
print(dupnds)
ages=[age(n,spT) for n in dupnds]
print(ages)



In [ ]:
print(rTrees['OG_1000'])

In [21]:
print(totdups)

defaultdict(<class 'int'>, {'Strpur': 8765, 'Ptyfla': 8408, 'Myxines': 3520, 'Eptbur': 3451, 'Letrei': 1339, 'Orylat': 2056, 'Galgal': 1828, 'Letjap': 5276, 'Deuterostomia': 3153, 'Bralan': 5374, 'Vertebrata': 4192, 'Cephalochordata': 7529, 'Euteleostei': 3094, 'Acapla': 2601, 'Gnathostomata': 5224, 'Homsap': 1368, 'Osteichthyes': 955, 'Sackow': 5280, 'Braflo': 2967, 'Musmus': 1859, 'Chipun': 2172, 'Mammalia': 911, 'Cyclostomata': 1990, 'Danrer': 3000, 'Hyperoartia': 2311, 'Hemichordata': 2379, 'Parata': 1761, 'Xentro': 3614, 'Brabel': 3281, 'Chordata': 711, 'Latcha': 1399, 'Lepocu': 739, 'Ceph2': 1090, 'Echinodermata': 772, 'Actinopterygii': 855, 'Leth': 1099, 'Chondrichthyes': 684, 'Ambulacraria': 766, 'Sarcopterygia': 287, 'Tetrapoda': 358, 'Calmil': 692, 'Petmar': 667, 'Amniotes': 200})


In [ ]:
i=0
for f,st in rTrees.items():
    pliv=[sst.name for sst in st.search_nodes(species='Parliv')]
    if len(pliv)<2: continue
    for g1,g2 in list(combinations(pliv,2)):
        anc=age(st.get_common_ancestor([g1,g2]),spT)
        gi1,gi2=g1.split('_',1)[1],g2.split('_',1)[1]
        m1,m2=gene2mod.get(gi1,'-'),gene2mod.get(gi2,'-')
        print(f,g1,g2,PlivNames.get(gi1,'-'),PlivNames.get(gi2,'-'),m1,m2,moddesc.get(m1,'-'),moddesc.get(m2,'-'),anc)
        
    i+=1
    #ages=[age(n,spT) for n in dupnds]
    #for a in list(set(ages)):
    #    totdups[a]+=1
    #dupNodes[f]=','.join(list(set(ages)))
    #print(f,dupnds,ages)
    if i==20: break

In [127]:
print(rTrees['OG_10015'].get_ascii(attributes=["name","D"], show_internal=True))


                     /-Lytvar_L_var_25220-RA, N
                /70, N
               |    |      /-Strpur_LOC589280, N
           /66, N    \n11, Y
          |    |           \-Strpur_LOC115923326, N
          |    |
          |     \-Parliv_PL20127, N
          |
      /95, Y              /-Parliv_PL19596, N
     |    |          /67, Y
     |    |         |     \-Parliv_PL20177, N
     |    |     /17, Y
     |    |    |    |      /-Parliv_PL37751, N
     |    |    |     \100, Y
-n73, N    \29, N          \-Parliv_PL19959, N
     |         |
     |         |     /-Lytvar_L_var_25221-RA, N
     |          \49, N
     |               \-Strpur_LOC763046, N
     |
      \-Auraur_scaffold1886_g2, N


In [35]:
i=0
gDupAge={}
for f,st in rTrees.items():
    #print(f)
    pliv=[sst.name for sst in st.search_nodes(species='Lepocu')]
    for l in st.search_nodes(species="Lepocu"):
        gid=l.name.split('_',1)[1]
        gda='Bila'
        for n in l.iter_ancestors():
            if n.D=='Y':
                #print(n.name,n.D,age(n,spT))
                gda=age(n,spT)
        gDupAge[gid]=gda
            
    i+=1
    #ages=[age(n,spT) for n in dupnds]
    #for a in list(set(ages)):
    #    totdups[a]+=1
    #dupNodes[f]=','.join(list(set(ages)))
    #print(f,dupnds,ages)
    #if i==3: break    
print(Counter(gDupAge.values()))

Counter({'Bila': 4898, 'Vertebrata': 4809, 'Gnathostomata': 2974, 'Deuterostomia': 2494, 'Chordata': 718, 'Osteichthyes': 271, 'Actinopterygii': 236, 'Lepocu': 147})


In [26]:
len(Counter(gDupAge.values()))

7

In [36]:
selDupCl=['Bila','Deuterostomia','Chordata','Vertebrata','Cyclostomata','Myxines','Parata']

In [55]:
selDupCl=['Bila','Deuterostomia','Chordata','Vertebrata','Gnathostomata','Osteichthyes','Actinopterygii','Lepocu']

## Importing WGCNA information

In [4]:
import csv

In [38]:
gene2mod={}
mod2gene={}
for f in glob("/Users/fmarletaz/Dropbox/Genomes/Hagfish/Hagfish_analysis/Gar_WGCNA/Gene_lists_in_Modules/non_annotated/*.txt"):
    mod=f.split('/')[-1].split('.')[0]
    genes=[]
    for l in open(f):
        gene2mod[l.strip()]=mod
        genes.append(l.strip())
    mod2gene[mod]=genes  
    

In [ ]:
geneNames={}

In [48]:
gene2mod={}
mod2gene={}
for f in glob("/Users/fmarletaz/Dropbox/Genomes/Hagfish/Hagfish_analysis/Hagfish_WGCNA/log2fpkm1/Gene_lists_in_Modules/eb_gid_only/*.txt"):
    mod=f.split('/')[-1].split('.')[0]
    genes=[]
    for l in open(f):
        gene2mod[l.strip()]=mod
        genes.append(l.strip())
    mod2gene[mod]=genes  
    

In [137]:
moddesc={}
for l in open("/Users/fmarletaz/Dropbox/Genomes/Urchin/Annotation/Pliv_WGCNA_log2fpkm1/test2.2_filtered_gene_models_less_clusters/cluster_desc.txt"):
    mod,desc=l.strip().split()
    moddesc[mod]=desc

In [152]:
modstra=[]
gbstr={}
for m in mod2gene:
    cnts=Counter([genephylo.get(g,'Parliv') for g in mod2gene[m]])
    gbstr[m]={c:cnts[c] for c in selclades}
    modstra.append([m,moddesc[m]]+[cnts[c] for c in selclades])

In [56]:
moddupa=[]
gbdpa={}
for m in mod2gene:
    cnts=Counter([gDupAge.get(g,'Lepocu') for g in mod2gene[m]])
    gbdpa[m]={c:cnts[c] for c in selDupCl}
    moddupa.append([m]+[cnts[c] for c in selDupCl])

In [57]:
moddupa

[['paleturquoise', 53, 38, 5, 64, 41, 6, 4, 25],
 ['mediumorchid', 12, 13, 3, 15, 11, 1, 0, 20],
 ['yellow4', 23, 8, 1, 18, 17, 1, 0, 4],
 ['lightcyan', 150, 40, 4, 62, 60, 8, 4, 15],
 ['brown4', 64, 18, 4, 24, 16, 3, 1, 7],
 ['salmon2', 6, 8, 2, 11, 4, 0, 0, 8],
 ['yellow', 107, 99, 16, 106, 104, 3, 6, 270],
 ['maroon', 30, 7, 5, 23, 18, 4, 2, 11],
 ['blue', 226, 205, 92, 532, 241, 14, 14, 523],
 ['pink', 213, 56, 13, 102, 82, 4, 3, 21],
 ['blue2', 8, 8, 4, 11, 9, 0, 0, 11],
 ['skyblue', 130, 20, 4, 46, 27, 5, 2, 14],
 ['darkorange2', 33, 18, 5, 37, 24, 3, 1, 18],
 ['black', 40, 71, 16, 71, 86, 4, 10, 213],
 ['plum1', 82, 14, 2, 42, 38, 6, 1, 9],
 ['thistle1', 50, 11, 2, 20, 17, 3, 1, 9],
 ['brown2', 8, 11, 1, 14, 2, 0, 2, 15],
 ['salmon4', 20, 10, 9, 37, 14, 0, 0, 15],
 ['ivory', 57, 21, 4, 41, 24, 5, 1, 21],
 ['bisque4', 34, 10, 4, 34, 31, 0, 1, 17],
 ['plum2', 17, 18, 6, 18, 14, 1, 2, 43],
 ['thistle2', 22, 21, 8, 24, 12, 2, 1, 27],
 ['darkgrey', 23, 32, 11, 62, 24, 2, 4, 122],
 ['

In [ ]:
modstra #['grey60', 'blastula_gastrula_pluteus_not_prism', 409, 40, 3, 1, 1, 1, 0, 9]

#### Hypergeometric test on origin (strata) age

In [ ]:
M=26054
n=tbc['Ambulacraria']
N=len()
stats.scipy.stats.hypergeom(M, n, N)

In [157]:
sum([sum(e.values()) for e in gbstr.values()])


26054

In [273]:
tbc=defaultdict(int)
for e in gbstr.values():
    for c in selclades:
        tbc[c]+=e[c]

In [284]:
hypres=[]
for m,e in gbstr.items():
    pv=[]
    #print(e)
    for c in selclades:
        hpd=stats.hypergeom(26054, tbc[c],sum(e.values()))
        pv.append(hpd.pmf(e[c]))
    hypres.append([m,moddesc[m]]+pv)
    

In [139]:
modstrati=pd.DataFrame(modstra, columns = ['mod', 'mod_desc']+selclades)  

In [288]:
modstrati.to_csv('strati_wgcna_nb.txt',sep='\t')

In [285]:
modstratihyp=pd.DataFrame(hypres, columns = ['mod', 'mod_desc']+selclades)  

In [287]:
modstratihyp.to_csv('strati_wgcna_phyp.txt',sep='\t')

#### Hypergeometric test on duplication ages

In [ ]:
gbdpa



In [58]:
tbd=defaultdict(int)
for e in gbdpa.values():
    for c in selDupCl:
        tbd[c]+=e[c]

In [59]:
hypresa=[]
for m,e in gbdpa.items():
    pv=[]
    #print(e)
    for c in selDupCl:
        hpd=stats.hypergeom(31313, tbd[c],sum(e.values()))
        pv.append(hpd.pmf(e[c]))
    hypresa.append([m]+pv)

In [60]:
tbd

defaultdict(int,
            {'Bila': 4852,
             'Deuterostomia': 2440,
             'Chordata': 707,
             'Vertebrata': 4754,
             'Gnathostomata': 2954,
             'Osteichthyes': 261,
             'Actinopterygii': 236,
             'Lepocu': 5846})

In [61]:
modduplage=pd.DataFrame(moddupa, columns = ['mod']+selDupCl)

In [63]:
modduplage.to_csv('duplage_wgcna_gar_nb.txt',sep='\t')

In [64]:
modduplahyp=pd.DataFrame(hypresa, columns = ['mod']+selDupCl)

In [65]:
modduplahyp.to_csv('duplage_wgcna_gar_phyp.txt',sep='\t')

In [155]:
igl=[[g,PlivNames.get(g,'-'),genephylo.get(g,'Parliv'),m,moddesc[m]] for g,m in gene2mod.items()]

In [156]:
gstrmod=pd.DataFrame(igl, columns = ['gid', 'gname','Strata','Module','ModuleDesc'])  

In [ ]:
selclades=['Eumetazoa','Bilateria','Deuterostomia','Ambulacraria', 'Echinodermata','Eleutherozoa','Echinozoa','Echinoidea']

In [291]:
#gstrmod[gstrmod.Strata.str.contains('Echinozoa')]#.to_csv('novel_in_echinoidea.txt',sep='\t')
gstrmod[gstrmod.Module==('lightyellow')]#.to_csv('novel_in_echinoidea.txt',sep='\t')

,gid,gname,Strata,Module,ModuleDesc
3574,PL00006,-,Parliv,lightyellow,lantern
3575,PL00167,unchar-20864,Echinoidea,lightyellow,lantern
3576,PL00185,unchar-14422,Echinoidea,lightyellow,lantern
3577,PL00216,unchar-10254,Bilateria,lightyellow,lantern
3578,PL00263,aces-2,Eumetazoa,lightyellow,lantern
...,...,...,...,...,...
3923,PL41273,unchar-21226,Eumetazoa,lightyellow,lantern
3924,PL41310,unchar-13618,Eumetazoa,lightyellow,lantern
3925,PL41327,rtbs-36,Eumetazoa,lightyellow,lantern
3926,PL41386,unchar-23794,Eumetazoa,lightyellow,lantern


In [292]:
gstrmod.to_csv('genes_wgcna_strata.txt',sep='\t')

In [ ]:
for col in cols:
    col_zscore = col + '_zscore'
    df[col_zscore] = (df[col] - df[col].mean())/df[col].std(ddof=0)

In [ ]:
modstratihyp

## Output